Importing the libraries


In [ ]:
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
import shutil
from sklearn.model_selection import train_test_split

Setting parameters

In [6]:
runs_dir = "Runs/"
model_name = "Hand_written_CNN_model"
model_dir = runs_dir + model_name + "/"
last_epoch = -1
Epochs = 30
train_batch_size = 32
test_batch_size = 16
img_width, img_height = 256, 256 

Splitting data in 3 parts(train, validation, test)

In [ ]:
# Path to dataset
dataset_path = './plantvillage dataset/color'

# Path to splited output
output_path = './split1'

classes = os.listdir(dataset_path)

# Parameters
test_size = 0.05  # 5% for test set
val_size = 0.12  # 12% of the remaining 80% for the validation set

# Function to copy files
def copy_files(files, source, dest):
    os.makedirs(dest, exist_ok=True)
    for file in files:
        file_path = os.path.join(source, file)
        if os.path.isfile(file_path):  # Ensure it's a file, not a directory
            shutil.copy(file_path, dest)

# Splitting and copying the dataset
for class_name in tqdm(classes):
    class_path = os.path.join(dataset_path, class_name)
    images = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]

    # Splitting
    train_val, test = train_test_split(images, test_size=test_size, random_state=42)
    train, val = train_test_split(train_val, test_size=val_size, random_state=42)

    # Copying files
    for dataset_type, dataset_files in zip(['train', 'val', 'test'], [train, val, test]):
        dest_path = os.path.join(output_path, dataset_type, class_name)
        copy_files(dataset_files, class_path, dest_path)

print("Dataset successfully split into training, validation, and test sets.")

Printing number of data points in each class

In [ ]:
ds_path = './split/train'

classes = sorted(os.listdir(ds_path))
num_classes = len(classes)
l =[]

for i in range(len(classes)):
    clas = classes[i]
    l.append((len(os.listdir(f"{ds_path}/{clas}")),clas))

l.sort(key=lambda x: x[0])

for i in l:
     print(i)

Creating and Compiling a simple CNN Classifier with dialation

In [ ]:
FACTOR = 1
x_inp = tf.keras.layers.Input(shape=(img_width, img_height, 3))
padding_layer = tf.keras.layers.ZeroPadding2D(padding=(2, 2))(x_inp) # 184
c1 = tf.keras.layers.Conv2D(int(24 * FACTOR), (3, 3), padding="same", activation=tf.nn.leaky_relu)(padding_layer) # 184
c_dial_1 = tf.keras.layers.Conv2D(int(8 * FACTOR), (3, 3), dilation_rate=1, padding="same", activation=tf.nn.leaky_relu)(padding_layer)
c1_out = tf.keras.layers.Concatenate(axis = 3)([c1, c_dial_1])
c2 = tf.keras.layers.Conv2D(int(12 * FACTOR), (3, 3), padding="same", activation=tf.nn.leaky_relu)(c1_out) # 184
c_dial_2 = tf.keras.layers.Conv2D(int(4 * FACTOR), (3, 3), dilation_rate=1, padding="same", activation=tf.nn.leaky_relu)(c1_out)
c2_out = tf.keras.layers.Concatenate(axis = 3)([c2, c_dial_2])
c3 = tf.keras.layers.Conv2D(int(12 * FACTOR), (3, 3), padding="same", activation=tf.nn.leaky_relu)(c2_out) # 184
c_dial_3 = tf.keras.layers.Conv2D(int(4 * FACTOR), (3, 3), dilation_rate=1, padding="same", activation=tf.nn.leaky_relu)(c2_out)
c3_out = tf.keras.layers.Concatenate(axis = 3)([c3, c_dial_3])
c4 = tf.keras.layers.Conv2D(int(12 * FACTOR), (3, 3), padding="same", activation=tf.nn.leaky_relu)(c3_out) # 184
c_dial_4 = tf.keras.layers.Conv2D(int(4 * FACTOR), (3, 3), dilation_rate=1, padding="same", activation=tf.nn.leaky_relu)(c3_out)
c4_out = tf.keras.layers.Concatenate(axis = 3)([c4, c_dial_4])
c5 = tf.keras.layers.Conv2D(int(12 * FACTOR), (3, 3), padding="same", activation=tf.nn.leaky_relu)(c4_out) # 184
c_dial_5 = tf.keras.layers.Conv2D(int(4 * FACTOR), (3, 3), dilation_rate=1, padding="same", activation=tf.nn.leaky_relu)(c4_out)
c5_out = tf.keras.layers.Concatenate(axis = 3)([c5, c_dial_5])
p1 = tf.keras.layers.MaxPooling2D()(c5_out)
c5 = tf.keras.layers.Conv2D(int(16 * FACTOR), (3, 3), padding="same", activation=tf.nn.leaky_relu)(p1) # 92
c6 = tf.keras.layers.Conv2D(int(16 * FACTOR), (3, 3), padding="same", activation=tf.nn.leaky_relu)(c5) # 92
p2 = tf.keras.layers.MaxPooling2D()(c6)
c7 = tf.keras.layers.Conv2D(int(32 * FACTOR), (3, 3), padding="same", activation=tf.nn.leaky_relu)(p2) # 46
c8 = tf.keras.layers.Conv2D(int(32 * FACTOR), (3, 3), padding="same", activation=tf.nn.leaky_relu)(c7) # 46
p3 = tf.keras.layers.MaxPooling2D()(c8)
c9 = tf.keras.layers.Conv2D(int(64 * FACTOR), (3, 3), padding="same", activation=tf.nn.leaky_relu)(p3) # 23
c10 = tf.keras.layers.Conv2D(int(64 * FACTOR), (3, 3), padding="same", activation=tf.nn.leaky_relu)(c9) # 23
p4 = tf.keras.layers.MaxPooling2D()(c10)
c11 = tf.keras.layers.Conv2D(int(128 * FACTOR), (3, 3), padding="same", activation=tf.nn.leaky_relu)(p4) # 12
c12 = tf.keras.layers.Conv2D(int(128 * FACTOR), (3, 3), padding="same", activation=tf.nn.leaky_relu)(c11) # 12
p7 = tf.keras.layers.GlobalMaxPooling2D()(c12)

d1 = tf.keras.layers.Dense(64, activation=tf.nn.leaky_relu)(p7)
d2 = tf.keras.layers.Dense(32, activation=tf.nn.leaky_relu)(d1)
out_layer = tf.keras.layers.Dense(num_classes, activation="softmax")(d2)
model = tf.keras.models.Model(inputs=[x_inp], outputs=[out_layer])
model.compile(optimizer=tf.keras.optimizers.Adam(0.0004), metrics=['accuracy'],loss=tf.keras.losses.categorical_crossentropy)

In [ ]:
model.summary()

Writing a data generator which yields batch size of data and labels

In [4]:
def datagen(ds_path,b):
    
    classes = sorted(os.listdir(ds_path))
    num_classes = len(classes)
    l =[]
    names = [sorted(os.listdir(f'{ds_path}/{i}')) for i in classes]
    for i in range(len(classes)):
        clas = classes[i]
        l.append((len(os.listdir(f"{ds_path}/{clas}")),clas))
    remaining = [x[0] for x in l]
    while 1:
        for i in range(num_classes):
            images = np.zeros((b,256,256,3))
            labels = np.zeros((b,num_classes))
            if remaining[i]>b:
                for j in range(b):
                    images[j] = plt.imread(f"{ds_path}/{classes[i]}/{names[i][j+l[i][0]-remaining[i]]}")[:,:,0:3]
                    labels[j][i] = 1 
                remaining[i] -= b
                
            else:
                remaining[i] = l[i][0]
                for j in range(b):
                    images[j] = plt.imread(f"{ds_path}/{classes[i]}/{names[i][j+l[i][0]-remaining[i]]}")[:,:,0:3]
                    labels[j][i] = 1
                remaining[i] -= b
        
            yield images, labels

Creating generator objects

In [5]:
train = datagen('./split/train/',train_batch_size)
val = datagen('./split/val/',train_batch_size)
test = datagen('./split/test/',test_batch_size)

Training loop

In [ ]:
for e in range(last_epoch + 1, Epochs):
    print("Epoch", e + 1, "/", Epochs, ":")

    model.fit(x = train, batch_size = train_batch_size ,validation_data= val, epochs = 1,steps_per_epoch=1000 , shuffle = True, validation_steps=20)

    model.save(model_dir + model_name + "_" + str(e) + ".h5")

Evaluation

In [ ]:
model_trained =  tf.keras.models.load_model("./Runs/Hand_written_CNN_model/Hand_written_CNN_model_6.h5")

In [ ]:
model_trained.evaluate(x= test, batch_size=train_batch_size, steps=54)

An example of a pretrained model fine tuning

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator


base_model = ResNet50(weights='imagenet', include_top=False)


for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)  

model_res = Model(inputs=base_model.input, outputs=predictions)

model_res.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_res.summary()